In [89]:
import pandas as pd
import numpy as np
import pickle

In [90]:
#url = 'https://raw.githubusercontent.com/MariaRosendal/Enhancing-Price-Momentum-with-RNN/main/'
url= ""

## 1. Importing input and output variables

In [91]:
# Import X 
X_train = pickle.load(open('X_traincum.pkl', 'rb' ))
X_test = pickle.load(open('X_testcum.pkl', 'rb' ))

In [92]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [93]:
X_train.shape, X_test.shape

((270000,), (270000,))

In [99]:
X_test= X_test.reshape(-1, 1)
X_train=  X_train.reshape(-1, 1)

In [100]:
# Import X for MLP and RF 
#X_train_mlp = pickle.load(open('X_train_vec.pkl', 'rb' ))
#X_test_mlp = pickle.load(open('X_test_vec.pkl', 'rb' ))

In [101]:
# Checking for NaN values
print(np.count_nonzero(np.isnan(X_train)), np.count_nonzero(np.isnan(X_test)))

0 0


In [102]:
X_train.shape, X_test.shape, 

((270000, 1), (270000, 1))

In [103]:
# Importing y
y = pd.read_csv(url+'y_reg.csv')
y['date'] =  pd.to_datetime(y['date'])
y = y.set_index(['date'])

In [104]:
# Defining length of y_train and y_test
y_train = y[y.index <= '1975-12-01']
y_test = y[y.index > '1975-12-01']

In [105]:
y_train.shape, y_test.shape

((270000, 1), (270000, 1))

In [106]:
# Reformating y
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()
y_train.shape, y_test.shape

((270000,), (270000,))

## 2. Models

In [107]:
# Importing packages
import tensorflow as tf
import matplotlib.pyplot as plt

import math
import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import BatchNormalization 
from keras.layers.core import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import r2_score

### Random Forest

In [108]:
from skopt.space import Real, Integer
from skopt import BayesSearchCV

In [109]:
# Ensuring data keeps temporal element when split
tscv = TimeSeriesSplit(n_splits=9) 

In [110]:
from sklearn.ensemble import RandomForestRegressor
import sklearn

In [111]:
m_rf = RandomForestRegressor()

In [112]:
n_estimators = 300

In [113]:
#parameters_rf = {'n_estimators':[50, 100, 200, 300], 'max_depth':[3, 5, 7], 'min_samples_split':[3, 5, 10, 50]}
param_opt = dict(max_depth=Integer(1,10))

In [114]:
bayesian = BayesSearchCV(
     m_rf,
     param_opt,
     n_iter=32,
     random_state=500,
     cv=tscv,
     verbose = 0, n_jobs=1
 )

bayesian_result = bayesian.fit(X_train, y_train)

/opt/anaconda3/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/anaconda3/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/anaconda3/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/anaconda3/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/anaconda3/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/anaconda3/

In [115]:
# Best model parameters
print('According to BayesianOptimization the best parameters are : ')
print('max_depth : ' + str(bayesian_result.best_params_['max_depth']))
#print('max_features : ' + str(bayesian_result.best_params_['max_features']))

According to BayesianOptimization the best parameters are : 
max_depth : 2


KeyError: 'max_features'

In [116]:
# Selecting best model
print("Best: %f using %s" % (bayesian_result.best_score_, bayesian_result.best_params_))
means = bayesian_result.cv_results_['mean_test_score']
stds = bayesian_result.cv_results_['std_test_score']
params = bayesian_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.003826 using OrderedDict([('max_depth', 2)])
-0.006557 (0.005292) with: OrderedDict([('max_depth', 10)])
-0.004449 (0.005828) with: OrderedDict([('max_depth', 5)])
-0.004798 (0.005780) with: OrderedDict([('max_depth', 6)])
-0.004750 (0.005747) with: OrderedDict([('max_depth', 6)])
-0.004176 (0.006012) with: OrderedDict([('max_depth', 1)])
-0.004747 (0.005739) with: OrderedDict([('max_depth', 6)])
-0.004064 (0.005997) with: OrderedDict([('max_depth', 3)])
-0.004215 (0.005904) with: OrderedDict([('max_depth', 4)])
-0.006059 (0.005358) with: OrderedDict([('max_depth', 9)])
-0.004084 (0.005980) with: OrderedDict([('max_depth', 3)])
-0.003826 (0.006135) with: OrderedDict([('max_depth', 2)])
-0.004478 (0.005859) with: OrderedDict([('max_depth', 5)])
-0.003893 (0.006076) with: OrderedDict([('max_depth', 2)])
-0.003865 (0.006070) with: OrderedDict([('max_depth', 2)])
-0.004154 (0.005921) with: OrderedDict([('max_depth', 4)])
-0.004773 (0.005774) with: OrderedDict([('max_depth', 6)])
-

In [118]:
# Defining best model parameters
max_depth = bayesian_result.best_params_['max_depth']
#max_features = bayesian_result.best_params_['max_features']

In [124]:
#best_rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
best_rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)

best_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, n_estimators=300)

In [125]:
print("Random forest:\t\t\t" + str(best_rf.score(X_test_mlp, y_test)))

NameError: name 'X_test_mlp' is not defined

In [ ]:
rf_predict = best_rf.predict(X_test_mlp)

In [ ]:
print(r2_score(y_test, rf_predict))

In [ ]:
r2_df = pd.DataFrame()
r2_df['y'] = y_test
r2_df['y_hat'] = rf_predict
r2_df['difference'] = (r2_df['y']-r2_df['y_hat'])

In [ ]:
r2_df = pd.DataFrame()
r2_df['y'] = y_test
r2_df['y_hat'] = rf_predict
r2_df['difference'] = (r2_df['y']-r2_df['y_hat'])
r2_df['y_2'] = r2_df['y']**2
r2_df['dif2'] = r2_df['difference']**2
y_2 = r2_df['y_2'].sum() 
dif2 = r2_df['dif2'].sum()
print(1 - (dif2 / y_2))

In [ ]:
rf = pd.DataFrame(rf_predict)
rf.nunique()

In [50]:
pickle.dump(rf_predict, open('rf_predict_test.pkl', 'wb' ))